# Body Text Inverted Index with Pre-computed Phrases

**Important:** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

This notebook loads pre-computed phrases from `strong_phrases.pkl` instead of calculating PMI from scratch.

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
# Check cluster status
!gcloud dataproc clusters list --region us-central1

## Imports & Setup

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import math
import numpy as np
from google.cloud import storage
from contextlib import closing

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
# Check graphframes jar
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
spark

In [ ]:
# ==================================
# IMPORTANT: Change bucket_name to your bucket!
# ==================================
bucket_name = 'db204905756'  # <-- שנה לשם ה-bucket שלך

full_path = f"gs://{bucket_name}/"
paths = []

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
        paths.append(full_path + b.name)

print(f"Found {len(paths)} parquet files")

## Stopwords & Stemmer Setup

In [ ]:
# Initialize Porter Stemmer
STEMMER = PorterStemmer()

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became", "make", "made",
                    "new", "list", "district", "com", "began"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['"-]?\w){2,24}""", re.UNICODE)

print(f"Total stopwords: {len(all_stopwords)}")
print(f"Stemmer initialized: {type(STEMMER).__name__}")

## Load Data - Body Text

In [ ]:
# Load parquet files
parquetFile = spark.read.parquet(*paths)

# Select BODY TEXT (not title!)
doc_text_pairs = parquetFile.select("text", "id").rdd

print(f"Total documents: {parquetFile.count():,}")

In [ ]:
# Preview sample
sample = doc_text_pairs.take(2)
print(f"Sample doc ID: {sample[0][1]}")
print(f"Sample text (first 500 chars): {sample[0][0][:500]}...")

## 🔴 Load Pre-computed Phrases from GCS

**Instead of calculating PMI, we load the already computed phrases!**

In [ ]:
# Download strong_phrases.pkl from GCS
# Try different possible locations

possible_paths = [
    f'gs://{bucket_name}/phrases/strong_phrases.pkl',
    f'gs://{bucket_name}/strong_phrases.pkl',
    f'gs://{bucket_name}/body_stemmed_phrases_idx/strong_phrases_body_stemmed.pkl',
    f'gs://{bucket_name}/title_with_phrases_idx/strong_phrases.pkl'
]

print("Searching for strong_phrases.pkl in GCS...")
for gcs_path in possible_paths:
    result = !gsutil ls {gcs_path} 2>/dev/null
    if result and 'CommandException' not in str(result):
        print(f"✅ Found: {gcs_path}")
        !gsutil cp {gcs_path} strong_phrases.pkl
        break
else:
    print("\n❌ Could not find strong_phrases.pkl. Let's search the bucket:")
    !gsutil ls -r gs://$bucket_name/ | grep -i phrase

In [ ]:
# Load the phrases
with open('strong_phrases.pkl', 'rb') as f:
    strong_phrases = pickle.load(f)

print(f"✅ Loaded {len(strong_phrases):,} pre-computed phrases!")

# Show some examples
print("\nSample phrases:")
for i, phrase in enumerate(list(strong_phrases)[:20]):
    print(f"  {phrase[0]}_{phrase[1]}")

## Tokenization Functions (with Stemming + Phrases)

In [ ]:
def tokenize_with_phrases_stemmed(text, phrases_set, stopwords_set):
    """
    Tokenize text with stemming, replacing recognized phrases with single tokens.
    
    Example:
    "I live in new york city" -> ["live", "new_york", "citi"]  (stemmed!)
    """
    if text is None:
        return []
    
    # Extract and stem tokens
    raw_tokens = [token.group().lower() for token in RE_WORD.finditer(text)]
    tokens = [STEMMER.stem(t) for t in raw_tokens if t not in stopwords_set]
    
    if len(tokens) <= 1:
        return tokens
    
    # Merge phrases
    result = []
    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1:
            bigram = (tokens[i], tokens[i+1])
            if bigram in phrases_set:
                result.append(f"{tokens[i]}_{tokens[i+1]}")
                i += 2
                continue
        result.append(tokens[i])
        i += 1
    
    return result


# Test
test_text = "The United States and New York City are located in North America"
test_result = tokenize_with_phrases_stemmed(test_text, strong_phrases, all_stopwords)
print(f"Input: {test_text}")
print(f"Output (stemmed + phrases): {test_result}")

## Load InvertedIndex Module

In [ ]:
# Load InvertedIndex module
%cd -q /home/dataproc
!ls inverted_index_gcp.py

sc = spark.sparkContext
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0, SparkFiles.getRootDirectory())

from inverted_index_gcp import InvertedIndex

In [ ]:
# Broadcast phrases and stopwords to all workers
strong_phrases_broadcast = sc.broadcast(strong_phrases)
all_stopwords_broadcast = sc.broadcast(all_stopwords)

print(f"✅ Broadcasted {len(strong_phrases):,} phrases to all workers")

## Build Inverted Index Functions

In [ ]:
NUM_BUCKETS = 124
INDEX_DIR = "body_stemmed_phrases"  # Directory name for body index with stemming

def token2bucket_id(token):
    """Map token to bucket number"""
    return int(_hash(token), 16) % NUM_BUCKETS


def word_count_with_phrases_stemmed(text, doc_id):
    """
    Count term frequency for each stemmed token (including phrases) in document.
    """
    tokens = tokenize_with_phrases_stemmed(
        text, 
        strong_phrases_broadcast.value, 
        all_stopwords_broadcast.value
    )
    counts = Counter(tokens)
    return [(token, (doc_id, tf)) for token, tf in counts.items()]


def reduce_word_counts(unsorted_pl):
    """Sort posting list by doc_id"""
    return sorted(unsorted_pl, key=lambda x: x[0])


def calculate_df(postings):
    """Calculate document frequency for each token"""
    return postings.map(lambda token: (token[0], len(token[1])))


def partition_postings_and_write(postings, base_dir):
    """Partition and write posting lists to GCS"""
    bucket_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()
    
    def write_bucket(b_w_pl):
        bucket_id, word_posting_pairs = b_w_pl
        return InvertedIndex.write_a_posting_list(
            (bucket_id, list(word_posting_pairs)), base_dir, bucket_name
        )
    
    return bucket_rdd.map(write_bucket)

## Build the Index

In [ ]:
%%time
print("Building inverted index with stemming and phrases for body text...")
print("This will take a while for the full corpus...")

# Step 1: Word counts (stemmed + phrases)
word_counts = doc_text_pairs.flatMap(lambda x: word_count_with_phrases_stemmed(x[0], x[1]))

# Step 2: Create posting lists
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

# Step 3: Filter rare terms (helps reduce index size)
MIN_DF = 50  # Minimum document frequency
postings_filtered = postings.filter(lambda x: len(x[1]) >= MIN_DF)

# Step 4: Calculate df
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

print(f"Total unique tokens (with df >= {MIN_DF}): {len(w2df_dict):,}")

In [ ]:
%%time
# Write posting lists to GCS
print(f"Writing posting lists to {INDEX_DIR}...")
_ = partition_postings_and_write(postings_filtered, INDEX_DIR).collect()
print("Done writing posting lists!")

In [ ]:
# Collect all posting list locations
super_posting_locs = defaultdict(list)

for blob in client.list_blobs(bucket_name, prefix=INDEX_DIR):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

print(f"Collected posting locations for {len(super_posting_locs):,} tokens")

In [ ]:
# Create and save InvertedIndex
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict

# Save locally
inverted.write_index('.', 'index')

# Upload to GCS
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/body_stemmed_phrases_idx/{index_src}'
!gsutil cp $index_src $index_dst

print(f"Index saved to {index_dst}")

In [ ]:
# Verify index size
!gsutil ls -lh $index_dst

## Calculate Document Lengths

In [ ]:
def calc_doc_length_stemmed(doc_id, text):
    """Calculate document length in stemmed tokens (including phrases)"""
    tokens = tokenize_with_phrases_stemmed(
        text, 
        strong_phrases_broadcast.value, 
        all_stopwords_broadcast.value
    )
    return (doc_id, len(tokens))

# Calculate lengths
print("Calculating document lengths (stemmed)...")
doc_lengths_rdd = doc_text_pairs.map(lambda x: calc_doc_length_stemmed(x[1], x[0]))
doc_lengths_dict = doc_lengths_rdd.collectAsMap()

print(f"Calculated lengths for {len(doc_lengths_dict):,} documents")

# Stats
lengths = list(doc_lengths_dict.values())
print(f"Average doc length: {np.mean(lengths):.1f}")
print(f"Median doc length: {np.median(lengths):.1f}")
print(f"Max doc length: {max(lengths):,}")

In [ ]:
# Save document lengths
lengths_filename = 'body_doc_lengths_stemmed.pickle'

with open(lengths_filename, 'wb') as f:
    pickle.dump(doc_lengths_dict, f)

# Upload to GCS
lengths_dst = f'gs://{bucket_name}/body_stemmed_phrases_idx/{lengths_filename}'
!gsutil cp $lengths_filename $lengths_dst

print(f"Document lengths saved to {lengths_dst}")

## Also save the phrases to the index folder

In [ ]:
# Copy phrases file to the index folder for convenience
phrases_dst = f'gs://{bucket_name}/body_stemmed_phrases_idx/strong_phrases.pkl'
!gsutil cp strong_phrases.pkl $phrases_dst
print(f"Phrases also saved to {phrases_dst}")

## Verify All Files

In [ ]:
print("Files in body_stemmed_phrases_idx/:")
!gsutil ls -lh gs://$bucket_name/body_stemmed_phrases_idx/

In [ ]:
print("\nPosting list files in body_stemmed_phrases/:")
!gsutil ls gs://$bucket_name/body_stemmed_phrases/ | head -20
print("...")
!gsutil ls gs://$bucket_name/body_stemmed_phrases/ | wc -l
print("total files")

## Summary

### Files Created:

| File | Location | Description |
|------|----------|-------------|
| `index.pkl` | `body_stemmed_phrases_idx/` | Inverted index (posting locs + df) |
| `body_doc_lengths_stemmed.pickle` | `body_stemmed_phrases_idx/` | Document lengths for BM25 |
| `strong_phrases.pkl` | `body_stemmed_phrases_idx/` | Set of detected phrases |
| `*.bin` | `body_stemmed_phrases/` | Binary posting list files |

### Usage in search_frontend.py:

```python
from nltk.stem.porter import PorterStemmer

STEMMER = PorterStemmer()

# Load phrases
with open('strong_phrases.pkl', 'rb') as f:
    strong_phrases = pickle.load(f)

# Process query with same tokenization (stemming + phrases)
def process_query(query):
    return tokenize_with_phrases_stemmed(query, strong_phrases, all_stopwords)
```

In [ ]:
print("✅ Body text index with pre-computed phrases creation complete!")